In [10]:
import time

import pandas as pd
import dotenv
import os
import sys
import datetime
sys.path.append('./_01_Simulation/')

import _01_Simulation.LLM_Edge_Usability

In [11]:
# load .env file
dotenv.load_dotenv()

True

# 1. Edge usability

In [12]:
# Params (currently only considers openai models
approaches = ["zeroshot", "fewshot"]

In [13]:
# load evaluation csv
df_evaluation = pd.read_csv(os.path.join(os.getenv('RESOURCES'), 'EvaluationDataset.csv'), sep=";")

In [14]:
df_evaluation.head()

,action,edgeUsable
0,A person fell on the ground.,False
1,There is an unexpected construction on the usu...,False
2,Fallen branches from nearby trees.,False
3,Street cleaning crew sweeping debris.,False
4,The sidewalk is obstructed by a parked car.,False


In [15]:
# TODO
df_evaluation = df_evaluation.iloc[:10]

In [16]:
# Invoke GPT-3.5 to obtain the predictions for edge-usability

for approach in approaches:
    predictions = []
    for index, row in df_evaluation.iterrows():
        model_output = _01_Simulation.LLM_Edge_Usability.invoke_llm(row["action"], model_type="openai", approach=approach)
        prediction = _01_Simulation.LLM_Edge_Usability.parse_response(model_output)
        predictions.append(prediction)
    
    df_evaluation["prediction"] = predictions
    df_evaluation.to_csv(os.path.join(os.getenv('RESULTS'), f'eval-res-edge-usability-openai-{approach}-{datetime.datetime.now().isoformat()}.csv'), sep=";", index=False)

/Users/paulkoenig/WebstormProjects/medicar/_01_Simulation/LLM_Edge_Usability.py:397: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  edges_list = [(f'{row[0]}', f'{row[1]}') for _, row in df.iterrows()]
/Users/paulkoenig/WebstormProjects/medicar/_01_Simulation/LLM_Edge_Usability.py:397: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  edges_list = [(f'{row[0]}', f'{row[1]}') for _, row in df.iterrows()]
/Users/paulkoenig/WebstormProjects/medicar/_01_Simulation/LLM_Edge_Usability.py:397: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as l